In [1]:
import sys
import spacy
import time
from spacy.kb import KnowledgeBase
from spacy.ml.models import load_kb
from spacy.training import Example
import config
from config import SPACY_DATASET
from pathlib import Path
import obonet
import os
import pickle
import random
from dataset_creation.utils import get_synonyms_formatted
from spacy.util import minibatch, compounding


## Testing  NER

In [2]:
nlp = spacy.load("../data/spacy/model-best/")


In [7]:
text = "Sexual reproduction"

In [8]:
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)


reproduction 7 19 ONTO


In [14]:
text = """The formation of multicellular or syncytial spore via septations derived from meiosis. 
A multicellular or syncytial spore is a structure that can be used for dissemination, for survival of adverse 
conditions because of its heat and dessication resistance, and/or for reproduction."""
doc = nlp(text)
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)


In [16]:
text = """Any process that modulates the frequency, rate or extent of conidium formation, 
a process of producing non-motile spores, called conidia, via mitotic asexual reproduction in higher fungi. 
Conidia are haploid cells genetically identical to their haploid parent. They are produced by conversion of 
hyphal elements, or are borne on
 sporogenous cells on or within specialized structures termed conidiophores, and participate in dispersal of the fungus."""
doc = nlp(text)
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)


In [18]:
text = 'via mitotic asexual reproduction in higher fungi'
doc = nlp(text)
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)


In [20]:

text = 'obsolete thioredoxin pathway'
doc = nlp(text)
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)


thioredoxin pathway 9 28 ONTO
